# Train, hyperparameter tune with PyTorch

## Upload dataset to default datastore


In [1]:
import azureml
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.41


In [1]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [3]:
from azureml.core import Experiment

experiment = Experiment(workspace=ws, name='torchvision')

In [4]:
import os
import shutil

script_folder = './torchdetect'
os.makedirs(script_folder, exist_ok=True)

shutil.copy('coco_eval.py',script_folder)
shutil.copy('coco_utils.py',script_folder)
shutil.copy('engine.py',script_folder)
shutil.copy('transforms.py',script_folder)
shutil.copy('utils.py',script_folder)
shutil.copy('coco_utils.py',script_folder)
shutil.copy('train.py',script_folder)
shutil.copy('train_sub.py',script_folder)
shutil.copy('train_str.py',script_folder)

In [4]:
ds = ws.get_default_datastore()

In [5]:
ds.container_name

'azureml-blobstore-a29dc687-5001-4ee4-ac74-7c17b122f449'

In [6]:
ds.upload(src_dir='/datadrive/torchvisionOD/BuildData/JPEGImages', target_path='JPEGImages', overwrite=True, show_progress=True)
ds.upload(src_dir='/datadrive/torchvisionOD/BuildData/Annotations', target_path='Annotations', overwrite=True, show_progress=True)

Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555394345.3154316.jpg
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555394398.0650895.jpg
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555394496.6084578.jpg
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555394543.6131394.jpg
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555394616.1978638.jpg
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555394757.2243044.jpg
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555394833.7655847.jpg
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555394955.6576927.jpg
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555394961.1118712.jpg
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555394982.938294.jpg
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555394998.0659883.jpg
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555395119.981925.jpg
Uploading /datadrive/torchvisionOD/BuildData/JPEGImage

Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555445463.0967178.jpg, 63 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555445581.1598172.jpg
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555445461.96303.jpg, 64 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555445582.2952673.jpg
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555445468.8340764.jpg, 65 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555445583.4390707.jpg
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555445434.5049953.jpg, 66 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555445584.5781817.jpg
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555445441.4204085.jpg, 67 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555445585.7518868.jpg
Up

Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555456304.8257241.jpg, 144 files out of an estimated total of 948
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555456634.8894475.jpg, 145 files out of an estimated total of 948
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555456443.7500815.jpg, 146 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555524521.1944127.jpg
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555456580.4901884.jpg, 147 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555524536.9752326.jpg
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555524552.7530339.jpg
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555524564.091825.jpg
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555456479.3577626.jpg, 148 files out of an estimated total of 948
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555456597.485567.jp

Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555539264.139183.jpg, 221 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555539680.4048843.jpg
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555539386.5144234.jpg, 222 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555539690.686179.jpg
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555539322.3077557.jpg, 223 files out of an estimated total of 948
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555539330.4309413.jpg, 224 files out of an estimated total of 948
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555539308.226951.jpg, 225 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555539711.259563.jpg
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555539280.09745.jpg, 226 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/Bu

Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555541070.3070428.jpg
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555541086.2894175.jpg
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555540585.771624.jpg, 301 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555541101.0342777.jpg
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555539975.0110133.jpg, 302 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555541136.4623954.jpg
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555540568.2117772.jpg, 303 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555541523.7105207.jpg
Uploaded /datadrive/torchvisionOD/BuildData/JPEGImages/1555540643.8720295.jpg, 304 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/JPEGImages/1555541533.847269.jpg
Uploaded /datadrive/torchvisionOD/BuildD

...

Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1555802004.0570817.xml, 676 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1555976985.8038874.xml
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1555977018.1794908.xml
Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1555961194.4282863.xml, 677 files out of an estimated total of 948
Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1555961243.5263155.xml, 678 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1555977045.0207222.xml
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1555977062.9187968.xml
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1555977080.7848096.xml
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1555976777.0518467.xml
Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1555961281.5024087.xml, 679 files out of an estimated total of 948
Uploading /datadrive/torchvi

Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1555976688.9662328.xml, 750 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1555801959.3900788.xml
Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1555801973.8782895.xml, 751 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1555456861.2187676.xml
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1556246193.9240339.xml
Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1556246538.4533386.xml, 752 files out of an estimated total of 948
Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1555802084.350428.xml, 753 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1555539731.8709865.xml
Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1555801948.2644846.xml, 754 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/Annotations/155624561

Uploading /datadrive/torchvisionOD/BuildData/Annotations/1555541044.0906777.xml
Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1555541778.4838188.xml, 822 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1555395014.744697.xml
Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1556245971.4925635.xml, 823 files out of an estimated total of 948
Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1555801997.3495982.xml, 824 files out of an estimated total of 948
Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1555960955.4308746.xml, 825 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1556246336.2696013.xml
Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1555801915.9140894.xml, 826 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1555539559.3381991.xml
Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1555456214

Uploading /datadrive/torchvisionOD/BuildData/Annotations/1555445581.1598172.xml
Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1555801914.7931037.xml, 895 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1556246152.2601287.xml
Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1556245818.7370672.xml, 896 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1555960919.637114.xml
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1555540543.31386.xml
Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1555802111.134976.xml, 897 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1555802104.4170861.xml
Uploaded /datadrive/torchvisionOD/BuildData/Annotations/1555539659.8027601.xml, 898 files out of an estimated total of 948
Uploading /datadrive/torchvisionOD/BuildData/Annotations/1555976568.4544413.xml
Uploading /datadrive/torchvision

$AZUREML_DATAREFERENCE_5f296c454c3f446480550ddda8863431

## Create  AmlCompute¶ 

In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "fboyluamlgpuclus"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-06-12T18:54:09.191000+00:00', 'errors': None, 'creationTime': '2019-06-11T18:48:22.478352+00:00', 'modifiedTime': '2019-06-11T18:49:10.788499+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


## Create A Pytorch Estimator

In [7]:
script_folder = './torchdetect'
image_name = 'fboylu/torchdet'

In [8]:
from azureml.train.dnn import PyTorch

script_params = {
    '--data_path': ds.as_mount(),
    '--workers': 8,
    '--learning_rate' : 0.005,
    '--epochs' : 5,
    '--anchor_sizes' : '16,32,64,128,256,512',
    '--anchor_aspect_ratios' : '0.25,0.5,1.0,2.0',
    '--rpn_nms_thresh' : 0.5,
    '--box_nms_thresh' : 0.3,
    '--box_score_thresh' : 0.10    
}

estimator = PyTorch(source_directory=script_folder, 
                    script_params=script_params,
                    compute_target=compute_target,
                    entry_script='train_str.py',
                    use_docker=True,
                    custom_docker_image=image_name,
                    user_managed=True,
                    use_gpu=True)

framework_version is not specified, defaulting to version 1.1.


In [9]:
# estimator.run_config.environment.python.interpreter_path = '/data/anaconda/envs/torchdetectaml/bin/python'
# estimator.run_config.history.snapshot_project = False
estimator.run_config.environment.environment_variables["PYTHONPATH"] = "$PYTHONPATH:/cocoapi/PythonAPI/"

### Submit job

In [10]:
run = experiment.submit(estimator)
print(run)

Run(Experiment: torchvision,
Id: torchvision_1560369221_b26532e3,
Type: azureml.scriptrun,
Status: Queued)


In [11]:
from azureml.widgets import RunDetails

In [ ]:
RunDetails(run).show()

In [13]:
# to get more details of your run
print(run.get_details())

{'runId': 'torchvision_1560369221_b26532e3', 'target': 'fboyluamlgpuclus', 'status': 'Queued', 'properties': {'azureml.runsource': 'experiment', 'ContentSnapshotId': 'bae6e356-2d31-4b30-bc32-14a0016258b3'}, 'runDefinition': {'script': 'train_str.py', 'arguments': ['--data_path', '$AZUREML_DATAREFERENCE_workspaceblobstore', '--workers', '8', '--learning_rate', '0.005', '--epochs', '5', '--anchor_sizes', '16,32,64,128,256,512', '--anchor_aspect_ratios', '0.25,0.5,1.0,2.0', '--rpn_nms_thresh', '0.5', '--box_nms_thresh', '0.3', '--box_score_thresh', '0.1'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'fboyluamlgpuclus', 'dataReferences': {'workspaceblobstore': {'dataStoreName': 'workspaceblobstore', 'mode': 'Mount', 'pathOnDataStore': None, 'pathOnCompute': None, 'overwrite': False}}, 'jobName': None, 'maxRunDurationSeconds': None, 'nodeCount': 1, 'environment': {'name': 'Experiment torchvision Environment', 'version': 'Autosave_2019-06-11T18:

In [14]:
run.wait_for_completion(show_output=True)

RunId: torchvision_1560369221_b26532e3
Web View: https://mlworkspace.azure.ai/portal/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/fboyluamlrg/providers/Microsoft.MachineLearningServices/workspaces/fboyluamlws/experiments/torchvision/runs/torchvision_1560369221_b26532e3

Streaming azureml-logs/70_driver_log.txt

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth

0.0%
0.0%
0.0%
0.0%
0.0%
0.0%
0.0%
0.0%
0.0%
0.0%
0.1%
0.1%
0.1%
0.1%
0.1%
0.1%
0.1%
0.1%
0.1%
0.1%
0.1%
0.1%
0.1%
0.1%
0.1%
0.1%
0.1%
0.1%
0.1%
0.1%
0.1%
0.1%
0.2%
0.2%
0.2%
0.2%
0.2%
0.2%
0.2%
0.2%
0.2%
0.2%
0.2%
0.2%
0.2%
0.2%
0.2%
0.2%
0.2%
0.2%
0.2%
0.2%
0.2%
0.2%
0.3%
0.3%
0.3%
0.3%
0.3%
0.3%
0.3%
0.3%
0.3%
0.3%
0.3%
0.3%
0.3%
0.3%
0.3%
0.3%
0.3%
0.3%
0.3%
0.3%
0.3%
0.3%
0.4%
0.4%
0.4%
0.4%
0.4%
0.4%
0.4%
0.4%
0.4%
0.4%
0.4%
0.4%
0.4%
0.4%
0.4%
0.4%
0.4%
0.4%
0.4%
0.4%
0.4%
0.5%
0.5%
0.5%


14.4%
14.4%
14.4%
14.4%
14.4%
14.4%
14.4%
14.4%
14.4%
14.4%
14.4%
14.4%
14.4%
14.4%
14.4%
14.4%
14.5%
14.5%
14.5%
14.5%
14.5%
14.5%
14.5%
14.5%
14.5%
14.5%
14.5%
14.5%
14.5%
14.5%
14.5%
14.5%
14.5%
14.5%
14.5%
14.5%
14.5%
14.5%
14.6%
14.6%
14.6%
14.6%
14.6%
14.6%
14.6%
14.6%
14.6%
14.6%
14.6%
14.6%
14.6%
14.6%
14.6%
14.6%
14.6%
14.6%
14.6%
14.6%
14.6%
14.7%
14.7%
14.7%
14.7%
14.7%
14.7%
14.7%
14.7%
14.7%
14.7%
14.7%
14.7%
14.7%
14.7%
14.7%
14.7%
14.7%
14.7%
14.7%
14.7%
14.7%
14.7%
14.8%
14.8%
14.8%
14.8%
14.8%
14.8%
14.8%
14.8%
14.8%
14.8%
14.8%
14.8%
14.8%
14.8%
14.8%
14.8%
14.8%
14.8%
14.8%
14.8%
14.8%
14.8%
14.9%
14.9%
14.9%
14.9%
14.9%
14.9%
14.9%
14.9%
14.9%
14.9%
14.9%
14.9%
14.9%
14.9%
14.9%
14.9%
14.9%
14.9%
14.9%
14.9%
14.9%
14.9%
15.0%
15.0%
15.0%
15.0%
15.0%
15.0%
15.0%
15.0%
15.0%
15.0%
15.0%
15.0%
15.0%
15.0%
15.0%
15.0%
15.0%
15.0%
15.0%
15.0%
15.0%
15.1%
15.1%
15.1%
15.1%
15.1%
15.1%
15.1%
15.1%
15.1%
15.1%
15.1%
15.1%
15.1%
15.1%
15.1%
15.1%
15.1%
15.1%
15.1%
15.1%
15.1

35.3%
35.3%
35.3%
35.3%
35.3%
35.4%
35.4%
35.4%
35.4%
35.4%
35.4%
35.4%
35.4%
35.4%
35.4%
35.4%
35.4%
35.4%
35.4%
35.4%
35.4%
35.4%
35.4%
35.4%
35.4%
35.4%
35.4%
35.5%
35.5%
35.5%
35.5%
35.5%
35.5%
35.5%
35.5%
35.5%
35.5%
35.5%
35.5%
35.5%
35.5%
35.5%
35.5%
35.5%
35.5%
35.5%
35.5%
35.5%
35.5%
35.6%
35.6%
35.6%
35.6%
35.6%
35.6%
35.6%
35.6%
35.6%
35.6%
35.6%
35.6%
35.6%
35.6%
35.6%
35.6%
35.6%
35.6%
35.6%
35.6%
35.6%
35.6%
35.7%
35.7%
35.7%
35.7%
35.7%
35.7%
35.7%
35.7%
35.7%
35.7%
35.7%
35.7%
35.7%
35.7%
35.7%
35.7%
35.7%
35.7%
35.7%
35.7%
35.7%
35.8%
35.8%
35.8%
35.8%
35.8%
35.8%
35.8%
35.8%
35.8%
35.8%
35.8%
35.8%
35.8%
35.8%
35.8%
35.8%
35.8%
35.8%
35.8%
35.8%
35.8%
35.8%
35.9%
35.9%
35.9%
35.9%
35.9%
35.9%
35.9%
35.9%
35.9%
35.9%
35.9%
35.9%
35.9%
35.9%
35.9%
35.9%
35.9%
35.9%
35.9%
35.9%
35.9%
35.9%
36.0%
36.0%
36.0%
36.0%
36.0%
36.0%
36.0%
36.0%
36.0%
36.0%
36.0%
36.0%
36.0%
36.0%
36.0%
36.0%
36.0%
36.0%
36.0%
36.0%
36.0%
36.0%
36.1%
36.1%
36.1%
36.1%
36.1%
36.1%
36.1%
36.1%
36.1

56.4%
56.4%
56.5%
56.5%
56.5%
56.5%
56.5%
56.5%
56.5%
56.5%
56.5%
56.5%
56.5%
56.5%
56.5%
56.5%
56.5%
56.5%
56.5%
56.5%
56.5%
56.5%
56.5%
56.5%
56.6%
56.6%
56.6%
56.6%
56.6%
56.6%
56.6%
56.6%
56.6%
56.6%
56.6%
56.6%
56.6%
56.6%
56.6%
56.6%
56.6%
56.6%
56.6%
56.6%
56.6%
56.6%
56.7%
56.7%
56.7%
56.7%
56.7%
56.7%
56.7%
56.7%
56.7%
56.7%
56.7%
56.7%
56.7%
56.7%
56.7%
56.7%
56.7%
56.7%
56.7%
56.7%
56.7%
56.7%
56.8%
56.8%
56.8%
56.8%
56.8%
56.8%
56.8%
56.8%
56.8%
56.8%
56.8%
56.8%
56.8%
56.8%
56.8%
56.8%
56.8%
56.8%
56.8%
56.8%
56.8%
56.9%
56.9%
56.9%
56.9%
56.9%
56.9%
56.9%
56.9%
56.9%
56.9%
56.9%
56.9%
56.9%
56.9%
56.9%
56.9%
56.9%
56.9%
56.9%
56.9%
56.9%
56.9%
57.0%
57.0%
57.0%
57.0%
57.0%
57.0%
57.0%
57.0%
57.0%
57.0%
57.0%
57.0%
57.0%
57.0%
57.0%
57.0%
57.0%
57.0%
57.0%
57.0%
57.0%
57.0%
57.1%
57.1%
57.1%
57.1%
57.1%
57.1%
57.1%
57.1%
57.1%
57.1%
57.1%
57.1%
57.1%
57.1%
57.1%
57.1%
57.1%
57.1%
57.1%
57.1%
57.1%
57.1%
57.2%
57.2%
57.2%
57.2%
57.2%
57.2%
57.2%
57.2%
57.2%
57.2%
57.2%
57.2

76.8%
76.8%
76.8%
76.8%
76.8%
76.8%
76.8%
76.8%
76.8%
76.8%
76.8%
76.8%
76.8%
76.8%
76.8%
76.8%
76.8%
76.8%
76.8%
76.9%
76.9%
76.9%
76.9%
76.9%
76.9%
76.9%
76.9%
76.9%
76.9%
76.9%
76.9%
76.9%
76.9%
76.9%
76.9%
76.9%
76.9%
76.9%
76.9%
76.9%
76.9%
77.0%
77.0%
77.0%
77.0%
77.0%
77.0%
77.0%
77.0%
77.0%
77.0%
77.0%
77.0%
77.0%
77.0%
77.0%
77.0%
77.0%
77.0%
77.0%
77.0%
77.0%
77.0%
77.1%
77.1%
77.1%
77.1%
77.1%
77.1%
77.1%
77.1%
77.1%
77.1%
77.1%
77.1%
77.1%
77.1%
77.1%
77.1%
77.1%
77.1%
77.1%
77.1%
77.1%
77.1%
77.2%
77.2%
77.2%
77.2%
77.2%
77.2%
77.2%
77.2%
77.2%
77.2%
77.2%
77.2%
77.2%
77.2%
77.2%
77.2%
77.2%
77.2%
77.2%
77.2%
77.2%
77.3%
77.3%
77.3%
77.3%
77.3%
77.3%
77.3%
77.3%
77.3%
77.3%
77.3%
77.3%
77.3%
77.3%
77.3%
77.3%
77.3%
77.3%
77.3%
77.3%
77.3%
77.3%
77.4%
77.4%
77.4%
77.4%
77.4%
77.4%
77.4%
77.4%
77.4%
77.4%
77.4%
77.4%
77.4%
77.4%
77.4%
77.4%
77.4%
77.4%
77.4%
77.4%
77.4%
77.4%
77.5%
77.5%
77.5%
77.5%
77.5%
77.5%
77.5%
77.5%
77.5%
77.5%
77.5%
77.5%
77.5%
77.5%
77.5%
77.5%
77.5

...

Epoch: [0]  [  0/424]  eta: 0:25:38  lr: 0.000017  loss: 1.7231 (1.7231)  loss_classifier: 0.7268 (0.7268)  loss_box_reg: 0.0107 (0.0107)  loss_objectness: 0.6962 (0.6962)  loss_rpn_box_reg: 0.2894 (0.2894)  time: 3.6295  data: 0.5403  max mem: 3690
Epoch: [0]  [ 10/424]  eta: 0:14:48  lr: 0.000135  loss: 1.6020 (1.5214)  loss_classifier: 0.6002 (0.5686)  loss_box_reg: 0.0047 (0.0063)  loss_objectness: 0.6915 (0.6924)  loss_rpn_box_reg: 0.2766 (0.2541)  time: 2.1458  data: 0.0584  max mem: 5997
Epoch: [0]  [ 20/424]  eta: 0:13:27  lr: 0.000253  loss: 1.1495 (1.2841)  loss_classifier: 0.1811 (0.3477)  loss_box_reg: 0.0047 (0.0080)  loss_objectness: 0.6901 (0.6906)  loss_rpn_box_reg: 0.2327 (0.2378)  time: 1.9162  data: 0.0067  max mem: 5997
Epoch: [0]  [ 30/424]  eta: 0:13:14  lr: 0.000371  loss: 0.9835 (1.2017)  loss_classifier: 0.1067 (0.2716)  loss_box_reg: 0.0132 (0.0122)  loss_objectness: 0.6840 (0.6876)  loss_rpn_box_reg: 0.2054 (0.2302)  time: 1.9466  data: 0.0046  max mem: 6298


Epoch: [0]  [360/424]  eta: 0:02:06  lr: 0.004268  loss: 0.3667 (0.6351)  loss_classifier: 0.0683 (0.0925)  loss_box_reg: 0.0627 (0.0691)  loss_objectness: 0.0476 (0.2778)  loss_rpn_box_reg: 0.1930 (0.1957)  time: 1.9899  data: 0.0063  max mem: 6578
Epoch: [0]  [370/424]  eta: 0:01:46  lr: 0.004386  loss: 0.3395 (0.6273)  loss_classifier: 0.0606 (0.0917)  loss_box_reg: 0.0655 (0.0691)  loss_objectness: 0.0455 (0.2719)  loss_rpn_box_reg: 0.1540 (0.1946)  time: 1.9901  data: 0.0062  max mem: 6578
Epoch: [0]  [380/424]  eta: 0:01:26  lr: 0.004504  loss: 0.3582 (0.6205)  loss_classifier: 0.0658 (0.0910)  loss_box_reg: 0.0660 (0.0691)  loss_objectness: 0.0455 (0.2661)  loss_rpn_box_reg: 0.1805 (0.1944)  time: 1.9685  data: 0.0062  max mem: 6578
Epoch: [0]  [390/424]  eta: 0:01:06  lr: 0.004622  loss: 0.3785 (0.6128)  loss_classifier: 0.0520 (0.0897)  loss_box_reg: 0.0545 (0.0684)  loss_objectness: 0.0484 (0.2611)  loss_rpn_box_reg: 0.1836 (0.1935)  time: 1.8873  data: 0.0064  max mem: 6578


Epoch: [1]  [190/424]  eta: 0:07:35  lr: 0.005000  loss: 0.2896 (0.3326)  loss_classifier: 0.0449 (0.0569)  loss_box_reg: 0.0440 (0.0590)  loss_objectness: 0.0378 (0.0536)  loss_rpn_box_reg: 0.1503 (0.1630)  time: 1.9032  data: 0.0064  max mem: 6578
Epoch: [1]  [200/424]  eta: 0:07:17  lr: 0.005000  loss: 0.2856 (0.3290)  loss_classifier: 0.0449 (0.0561)  loss_box_reg: 0.0516 (0.0584)  loss_objectness: 0.0369 (0.0528)  loss_rpn_box_reg: 0.1393 (0.1617)  time: 1.9529  data: 0.0063  max mem: 6578
Epoch: [1]  [210/424]  eta: 0:06:58  lr: 0.005000  loss: 0.2273 (0.3255)  loss_classifier: 0.0370 (0.0557)  loss_box_reg: 0.0294 (0.0582)  loss_objectness: 0.0305 (0.0516)  loss_rpn_box_reg: 0.1377 (0.1600)  time: 2.0215  data: 0.0066  max mem: 6578
Epoch: [1]  [220/424]  eta: 0:06:38  lr: 0.005000  loss: 0.2559 (0.3239)  loss_classifier: 0.0442 (0.0558)  loss_box_reg: 0.0373 (0.0586)  loss_objectness: 0.0300 (0.0508)  loss_rpn_box_reg: 0.1171 (0.1588)  time: 1.9537  data: 0.0069  max mem: 6578


Epoch: [2]  [ 20/424]  eta: 0:13:07  lr: 0.005000  loss: 0.2219 (0.2426)  loss_classifier: 0.0379 (0.0434)  loss_box_reg: 0.0395 (0.0457)  loss_objectness: 0.0225 (0.0264)  loss_rpn_box_reg: 0.1228 (0.1271)  time: 1.9302  data: 0.0064  max mem: 6578
Epoch: [2]  [ 30/424]  eta: 0:12:59  lr: 0.005000  loss: 0.2219 (0.2392)  loss_classifier: 0.0372 (0.0401)  loss_box_reg: 0.0268 (0.0448)  loss_objectness: 0.0242 (0.0257)  loss_rpn_box_reg: 0.1337 (0.1286)  time: 1.9491  data: 0.0058  max mem: 6578


ExperimentExecutionException: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run

## Tune Model Hyperparameters

In [22]:
from azureml.train.dnn import PyTorch

script_params = {
    '--data_path': ds.as_mount(),
    '--workers': 8,
    '--epochs' : 10,
    '--box_nms_thresh' : 0.3,
    '--box_score_thresh' : 0.10    
}

estimator = PyTorch(source_directory=script_folder, 
                    script_params=script_params,
                    compute_target=compute_target,
                    entry_script='train_str.py',
                    use_docker=True,
                    custom_docker_image=image_name,
                    user_managed=True,
                    use_gpu=True)

estimator.run_config.environment.environment_variables["PYTHONPATH"] = "$PYTHONPATH:/cocoapi/PythonAPI/"

framework_version is not specified, defaulting to version 1.1.


In [23]:
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, uniform, choice, HyperDriveConfig, PrimaryMetricGoal

In [24]:
param_sampling = RandomParameterSampling( {
        'learning_rate': uniform(0.0005, 0.005),
        'rpn_nms_thresh': uniform(0.3, 0.7),
        'anchor_sizes': choice('16', '16,32', '16,32,64', '16,32,64,128', '16,32,64,128,256', '16,32,64,128,256,512'),
        'anchor_aspect_ratios': choice('0.25', '0.25,0.5', '0.25,0.5,1.0', '0.25,0.5,1.0,2.0')
    }
)

In [25]:
early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=2, delay_evaluation=2)

hyperdrive_config = HyperDriveConfig(estimator=estimator,
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     primary_metric_name='mAP@IoU=0.50',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=16,
                                     max_concurrent_runs=4)

In [26]:
hyperdrive_run = experiment.submit(hyperdrive_config)

In [29]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: torchvision_1560447191261
Web View: https://mlworkspace.azure.ai/portal/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/fboyluamlrg/providers/Microsoft.MachineLearningServices/workspaces/fboyluamlws/experiments/torchvision/runs/torchvision_1560447191261

Streaming azureml-logs/hyperdrive.txt

"<START>[2019-06-13T17:33:11.698316][API][INFO]Experiment created<END>\n""<START>[2019-06-13T17:33:12.307271][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2019-06-13T17:33:12.471081][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2019-06-13T17:33:13.7958414Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>
